# Building Robust APIs with FastAPI - Error Handling, Data Validation, and Custom Exceptions

## Overview
In this notebook, we will explore how to build robust APIs using **FastAPI** by implementing error handling, data validation, and custom exceptions. We'll cover how to handle scenarios like borrowing books, booking trips, updating user preferences, and managing spacecraft details. Additionally, we'll learn how to create custom exception handlers for better error management.



## Table of Contents
1. **Borrowing Books with Error Handling**
2. **Listing and Booking Trips**
3. **Updating User Preferences with Pydantic Models**
4. **Booking Trips with Advanced Error Handling**
5. **Returning JSON-Compatible Responses**
6. **Assignment: Custom Exception Handling for Match Tickets**

## 1. Borrowing Books with Error Handling

### Definition:
**Error Handling** is the process of managing unexpected or invalid inputs in an API. In FastAPI, you can use `HTTPException` to raise HTTP errors (e.g., 404 Not Found, 400 Bad Request) when certain conditions are not met. This ensures that your API provides meaningful error messages to clients.

### Example 1: Borrowing a Book

```python
from fastapi import FastAPI, HTTPException

app = FastAPI()

books = {
    "1": {"title": "Our Time", "borrowed": False},
    "2": {"title": "Into the Fire", "borrowed": False},
}

@app.get("/borrow-book/{book_id}")
async def borrow_book(book_id: str):
    """
    This endpoint allows users to borrow a book based on its ID.
    - `book_id`: A mandatory path parameter specifying the book's ID.
    """
    book = books.get(book_id)
    if not book:
        raise HTTPException(status_code=404, detail="Book not found")
    if book["borrowed"]:
        raise HTTPException(status_code=400, detail="Book is already borrowed")
    book["borrowed"] = True
    return {"message": f"You have successfully borrowed '{book['title']}'"}
```

#### Explanation:
- **Route**: `/borrow-book/{book_id}`
- **Parameters**:
  - `book_id`: A mandatory path parameter that identifies the book.
- **Functionality**:
  - The function checks if the book exists in the `books` dictionary.
  - If the book is not found, it raises a 404 error.
  - If the book is already borrowed, it raises a 400 error.
  - If the book is available, it marks it as borrowed and returns a success message.

## 2. Listing and Booking Trips

### Definition:
**Listing and Booking Trips** involves creating endpoints to retrieve available trips and book a specific trip. These endpoints often include metadata like tags, summaries, and descriptions to improve API documentation.

### Example 2: Listing and Booking Trips

```python
from fastapi import FastAPI

app = FastAPI()

trips = [
    {"id": 1, "destination": "Mars", "price": 10000},
    {"id": 2, "destination": "Moon", "price": 5000},
]

@app.get("/trips", tags=["Trips"], summary="List of trips",
         description="Retrieve a list of available interstellar trips with their details.")
async def list_trips():
    """
    This endpoint lists all available trips.
    """
    return trips

@app.post("/book-trip/{trip_id}", tags=["Booking"], summary="Book a trip",
          description="Book an interstellar trip by providing the trip ID.")
async def book_trip(trip_id: int):
    """
    This endpoint allows users to book a trip based on its ID.
    - `trip_id`: A mandatory path parameter specifying the trip's ID.
    """
    trip = next((trip for trip in trips if trip["id"] == trip_id), None)
    if trip is None:
        return {"error": "Trip not found"}
    return {"message": f"Trip to {trip['destination']} booked successfully!"}
```

#### Explanation:
- **Routes**:
  - `/trips`: Lists all available trips.
  - `/book-trip/{trip_id}`: Books a specific trip based on its ID.
- **Metadata**:
  - Tags, summaries, and descriptions are added to improve API documentation.
- **Functionality**:
  - The `/trips` endpoint returns a list of all available trips.
  - The `/book-trip/{trip_id}` endpoint checks if the trip exists and returns a success message if booked.

## 3. Updating User Preferences with Pydantic Models

### Definition:
**Pydantic Models** are used for data validation and serialization in FastAPI. They allow you to define the structure of incoming and outgoing data, enforce type constraints, and apply additional validation rules. This ensures that the data conforms to the expected format before it is processed.

### Example 3: Updating Home Design Preferences

```python
from fastapi import FastAPI
from fastapi.encoders import jsonable_encoder
from pydantic import BaseModel, Field
from typing import Optional

app = FastAPI()

class HomeDesign(BaseModel):
    color_scheme: Optional[str] = Field(None, example="Warm")
    materials: Optional[str] = Field(None, example="Wood")

user_preferences = {}

@app.put("/update-design/{user_id}", response_model=HomeDesign)
async def update_design(user_id: int, design: HomeDesign):
    """
    This endpoint updates a user's home design preferences.
    - `user_id`: A mandatory path parameter specifying the user's ID.
    - `design`: A Pydantic model representing the user's design preferences.
    """
    json_compatible_design = jsonable_encoder(design)
    user_preferences[user_id] = json_compatible_design
    return json_compatible_design
```

#### Explanation:
- **Pydantic Model**: `HomeDesign` defines the structure of the incoming data with optional fields for `color_scheme` and `materials`.
- **Route**: `/update-design/{user_id}`
- **Functionality**:
  - The function validates the incoming design data using the `HomeDesign` model.
  - It stores the validated data in the `user_preferences` dictionary and returns it.

## 4. Booking Trips with Advanced Error Handling

### Definition:
**Advanced Error Handling** involves managing multiple error scenarios, such as checking for trip availability and ensuring trips are not double-booked. This ensures that your API handles edge cases gracefully.

### Example 4: Booking Trips with Availability Checks

```python
from fastapi import FastAPI, HTTPException
from typing import Dict

app = FastAPI()

trips = {
    "1": {"destination": "Mars", "available": True},
    "2": {"destination": "Moon", "available": False},
}

bookings: Dict[str, str] = {}

@app.post("/book-trip/{trip_id}")
async def book_trip(trip_id: str):
    """
    This endpoint allows users to book a trip based on its ID.
    - `trip_id`: A mandatory path parameter specifying the trip's ID.
    """
    if trip_id not in trips:
        raise HTTPException(status_code=404, detail="Trip not found")
    if not trips[trip_id]["available"]:
        raise HTTPException(status_code=400, detail="Trip is not available")
    if trip_id in bookings:
        raise HTTPException(status_code=400, detail="Trip is already booked")
    bookings[trip_id] = "booked"
    return {"message": f"Trip to {trips[trip_id]['destination']} booked successfully!"}
```

#### Explanation:
- **Route**: `/book-trip/{trip_id}`
- **Parameters**:
  - `trip_id`: A mandatory path parameter that identifies the trip.
- **Functionality**:
  - The function checks if the trip exists, is available, and has not been booked.
  - If any condition fails, it raises an appropriate HTTP error.
  - If all conditions are met, it marks the trip as booked and returns a success message.

## 5. Returning JSON-Compatible Responses

### Definition:
**JSON-Compatible Responses** ensure that the data returned by your API is serialized into a format that can be easily consumed by clients. FastAPI provides the `jsonable_encoder` utility to convert Pydantic models and other objects into JSON-compatible formats.

### Example 5: Returning Spacecraft Details

```python
from fastapi import FastAPI
from fastapi.encoders import jsonable_encoder
from pydantic import BaseModel

app = FastAPI()

class SpacecraftDetails(BaseModel):
    name: str
    capacity: int

@app.get("/spacecraft/{name}")
async def get_spacecraft(name: str):
    """
    This endpoint returns details of a spacecraft.
    - `name`: A mandatory path parameter specifying the spacecraft's name.
    """
    spacecraft = SpacecraftDetails(name=name, capacity=200)
    json_compatible_spacecraft = jsonable_encoder(spacecraft)
    return json_compatible_spacecraft
```

#### Explanation:
- **Pydantic Model**: `SpacecraftDetails` defines the structure of the spacecraft data.
- **Route**: `/spacecraft/{name}`
- **Functionality**:
  - The function creates a `SpacecraftDetails` object and converts it into a JSON-compatible format using `jsonable_encoder`.
  - It returns the JSON-compatible spacecraft details.

## 6. Assignment: Custom Exception Handling for Match Tickets

### Definition:
**Custom Exception Handling** allows you to define and handle specific error scenarios in your API. By creating custom exception classes and handlers, you can provide more meaningful error messages and responses to clients.

### Example 6: Booking Match Tickets with Custom Exceptions

```python
from fastapi import FastAPI, HTTPException, Request, status
from fastapi.responses import JSONResponse

app = FastAPI()

# Mock data: match tickets availability (0 = sold out)
match_tickets = {"Portugal vs Argentina": 0}

# Custom exception class for specific API errors
class CustomAPIException(Exception):
    def __init__(self, status_code: int, message: str):
        self.status_code = status_code
        self.message = message

# Handles responses for CustomAPIException
@app.exception_handler(CustomAPIException)
async def custom_api_exception_handler(request: Request, exc: CustomAPIException):
    return JSONResponse(status_code=exc.status_code, content={"message": exc.message})

# Endpoint to book a match ticket
@app.post("/book-match-ticket")
async def book_match_ticket(match: str):
    """
    This endpoint allows users to book a match ticket.
    - `match`: A mandatory query parameter specifying the match name.
    """
    if match not in match_tickets:
        raise CustomAPIException(status.HTTP_422_UNPROCESSABLE_ENTITY, "Invalid booking request.")
    if match_tickets[match] == 0:
        raise CustomAPIException(status.HTTP_400_BAD_REQUEST, "Sorry, tickets for this match are sold out.")
    return {"message": f"Ticket for {match} booked successfully!"}
```

#### Explanation:
- **Custom Exception Class**: `CustomAPIException` defines custom error messages and status codes.
- **Route**: `/book-match-ticket`
- **Parameters**:
  - `match`: A mandatory query parameter that specifies the match name.
- **Functionality**:
  - The function checks if the match exists and if tickets are available.
  - If any condition fails, it raises a `CustomAPIException`.
  - The custom exception handler returns a JSON response with the error message and status code.